In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
        # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Define feature extractor

In [2]:
import fnmatch
from typing import Any, Dict, List, Tuple
from torch import nn
from PIL import Image
import ast 

class HuggingFaceFeatureExtractor:
    """
    A feature extractor for Hugging Face (or any PyTorch) models that captures
    intermediate activations from any layer specified by exact name or glob pattern.

    Example usage:
        from transformers import BertModel
        model = BertModel.from_pretrained("bert-base-uncased")
        # Specify layers (using glob patterns is supported)
        layers_to_extract = ["encoder.layer.*.output"]

        # Using the extractor as a context manager ensures hooks are removed automatically.
        with HuggingFaceFeatureExtractor(model, layers_to_extract, detach=True) as extractor:
            # Perform a forward pass as usual
            outputs = model(input_ids, attention_mask=mask)
            # Get a copy of the extracted features
            features = extractor.features
            # Now 'features' is a dict mapping layer names to their activation tensors.
    """

    def __init__(self, model: nn.Module, layers: List[str], detach: bool = True):
        self.model = model
        self.detach = detach
        # Expand layer patterns into full module names
        self.layers = self._expand_layers(model, layers)
        self._features: Dict[str, Any] = {}
        self._handles: Dict[str, Any] = {}
        self._register_hooks()

    def _register_hooks(self):
        """Register forward hooks on each specified layer."""
        for layer in self.layers:
            sub_module = self.model.get_submodule(layer)
            handle = sub_module.register_forward_hook(self._make_hook(layer))
            self._handles[layer] = handle

    def _make_hook(self, layer_name: str):
        def hook(module: nn.Module, inputs: Tuple[Any, ...], output: Any):
            # Optionally detach to break the graph and save memory.
            self._features[layer_name] = output.detach() if self.detach else output
        return hook

    def clear(self):
        """Clear the stored features before a new forward pass."""
        self._features.clear()

    @property
    def features(self) -> Dict[str, Any]:
        """Return a copy of the captured features."""
        return dict(self._features)

    def __call__(self, *args, **kwargs) -> Any:
        """
        Run the model forward. This automatically clears previous features,
        then performs a forward pass, capturing intermediate activations.
        Returns the model's original output.
        """
        self.clear()
        return self.model(*args, **kwargs)

    def remove_hooks(self):
        """Remove all registered hooks."""
        for handle in self._handles.values():
            handle.remove()
        self._handles.clear()

    def __enter__(self):
        """Enter context: hooks are already registered."""
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Exit context: remove all hooks."""
        self.remove_hooks()

    @staticmethod
    def _expand_layers(model: nn.Module, layers: List[str]) -> List[str]:
        """
        Expand a list of layer names and/or glob patterns to all matching module names
        in the given model. Raises an error if a specified name or pattern doesn't match.
        """
        all_layers = [name for name, _ in model.named_modules() if name]  # skip the root module ''
        all_layers_set = set(all_layers)
        expanded = []
        special_chars = set("*?[]")
        for layer in layers:
            if not any(char in layer for char in special_chars):
                if layer not in all_layers_set:
                    raise ValueError(f"Layer '{layer}' not found in the model.")
                expanded.append(layer)
            else:
                matches = fnmatch.filter(all_layers, layer)
                if not matches:
                    raise ValueError(f"No layers match the pattern '{layer}'.")
                expanded.extend(matches)
        return expanded


### Define functions for loading video/audio/transcript

In [3]:
import pandas as pd
import torchaudio
import cv2
import torch
import torch.nn.functional as F
from typing import Tuple, List, Callable




def load_transcript(
    path: str
) -> pd.DataFrame:
    """
    Loads a transcript file (TSV) into a pandas DataFrame.

    Parameters:
        path (str): Path to the transcript file.

    Returns:
        pd.DataFrame: DataFrame containing the transcript data.
    """
    try:
        df = pd.read_csv(path, sep='\t')
        return df
    except Exception as e:
        raise RuntimeError(f"Error loading transcript from {path}: {e}")


def load_audio(
    path: str,
    sampling_rate: int = 48000,
    stereo: bool = True
) -> (torch.Tensor, int):
    """
    Loads an audio file using torchaudio, converts the waveform to half precision,
    optionally converts stereo audio to mono, resamples it to the specified sampling_rate
    if needed, and returns the waveform and sample rate.

    Parameters:
        path (str): Path to the audio file.
        sampling_rate (int): Desired sampling rate for the output waveform.
        stereo (bool): If False, converts stereo audio to mono.

    Returns:
        tuple: (waveform_fp16, sampling_rate) where waveform_fp16 is a tensor in float16.
    """
    try:
        # Set the backend to 'ffmpeg' if available
        torchaudio.set_audio_backend("ffmpeg")
        waveform, orig_sr = torchaudio.load(path)
        
        # Convert to mono if stereo is False and the waveform has multiple channels
        if not stereo and waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        # Resample if original sample rate is different from the desired sampling rate
        if orig_sr != sampling_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=orig_sr, new_freq=sampling_rate)
            waveform = resampler(waveform)
        
        # Convert the waveform to half precision (float16)
        waveform_fp16 = waveform.half()
        del waveform
        return waveform_fp16, sampling_rate
    except Exception as e:
        raise RuntimeError(f"Error loading audio from {path}: {e}")


def load_video(
    path: str,
    resolution: Tuple[int, int] = (224, 224),
    tensor_dtype: torch.dtype = torch.float16,
    verbose: bool = True,
) -> torch.Tensor:
    """
    Loads a video file, reads its frames, converts each frame from BGR to RGB,
    resizes to 224x224, and returns a tensor containing all frames.

    Parameters:
        path (str): Path to the video file.

    Returns:
        torch.Tensor: Tensor of shape [num_frames, 3, 224, 224] containing the video frames.
    """
    cap = cv2.VideoCapture(path)

    if not cap.isOpened():
        raise IOError("Cannot open video file: {}".format(path))

    # Get video FPS and calculate number of frames for 10 seconds
    fps = cap.get(cv2.CAP_PROP_FPS)
    num_frames_to_read = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if verbose:
        print("Total number of frames in the video:", cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print("Original Resolution:", (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        print("FPS:", fps)
        print("Duration (seconds):", num_frames_to_read / fps)
        print("Target Resolution:", resolution)

    frames = torch.zeros(num_frames_to_read, 3, 224, 224, dtype=tensor_dtype)

    for i in range(num_frames_to_read):
        ret, frame = cap.read()

        if not ret:
            break

        # Optionally, convert the frame from BGR to RGB (if needed)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Convert the frame (numpy array) to a torch tensor and permute dimensions to [C, H, W]
        frame_tensor = torch.from_numpy(frame_rgb).permute(2, 0, 1) 
        # Resize the frame to 224x224
        frame_tensor = torch.nn.functional.interpolate(frame_tensor.unsqueeze(0), size=224, mode='bilinear', align_corners=False)
        frames[i] = frame_tensor

    cap.release()

    if verbose:
        print(f"Read {len(frames)} frames.")
        print(f"Frames shape: {frames.shape}")

    return frames, fps

In [4]:
# from transformers import BertModel, BertTokenizer
# import torch

# # Initialize the model and tokenizer.
# model = BertModel.from_pretrained("bert-base-uncased")
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# # Prepare an example input.
# text = "This is a test sentence for feature extraction."
# inputs = tokenizer(text, return_tensors="pt")

# # Define the layers to extract using a glob pattern.
# # For BERT, each encoder layer (0-indexed) has a submodule "output" that we target.
# layers_to_extract = ["encoder.layer.0.output"]

# # Use the feature extractor as a context manager so that hooks are cleaned up automatically.
# with HuggingFaceFeatureExtractor(model, layers_to_extract, detach=True) as extractor:
#     # Run a forward pass. The extractor clears previous features automatically.
#     outputs = extractor(**inputs)

#     # Retrieve the extracted features.
#     features = extractor.features

#     # Print out the shape of the main model output.
#     print("Main model output (last_hidden_state) shape:", outputs.last_hidden_state.shape)

#     # Iterate over the extracted features and print their shapes.
#     for layer_name, activation in features.items():
#         print(f"Layer: {layer_name}, Feature shape: {activation.shape}")


### Define main function for iterating over the files and write .h5 for each one 

In [5]:
from pathlib import Path
import h5py
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt



def extract_features(
    parts: List[str],
    movies_base: str,
    transcripts_base: str,
    output_dir: str,
    extraction_fn: Callable,
    interval: int = 1.49,
    verbose: bool = True,
    modality: str = 'all',
):
    """
    Extracts features from the specified parts of the dataset using the provided extraction function.

    Parameters:
        parts (List[str]): List of parts to extract features from. This is the subdirectory name under friends and movie10 folders.
        movies_base (str): Path to the base directory containing movie files.
        transcripts_base (str): Path to the base directory containing transcript files.
        output_dir (str): Path to the output directory where features will be saved.
        interval (int): Interval (in seconds) at which to extract features. Default is 1.49 seconds (the TR for the dataset).
        extraction_fn (function): Function that extracts features from the stimuli. The function should take the following arguments:
            - video: torch.Tensor containing video frames (num_frames, 3, 224, 224)
            - audio: torch.Tensor containing audio waveform (2, num_samples)
            - transcript: array containing strings of words (num_words,)
            - verbose: bool indicating whether to print verbose output.
            and should return a dictionary mapping layer names to extracted features as torch.Tensor.
        verbose (bool): Whether to print verbose output.
        modality (str): Modality to extract features from. Default is 'all'. Options are 'video', 'audio', 'transcript'.


    """
    global video_section_g, audio_section_g, transcript_section_g

    movies_base = Path(movies_base)
    transcripts_base = Path(transcripts_base)

    # Verify that the base directories exist.
    if not movies_base.exists():
        raise FileNotFoundError(f"Movies directory not found: {movies_base}")
    if not transcripts_base.exists():
        raise FileNotFoundError(f"Transcripts directory not found: {transcripts_base}")

    # Iterate through all directories under movies_base.
    for folder in movies_base.rglob('*'):
        if folder.is_dir() and folder.name in parts:
            # Iterate through mkv files in the matched directory.
            for movie_file in folder.glob('*.mkv'):
                # Compute the relative path from movies_base.
                try:
                    rel_folder = folder.relative_to(movies_base)
                except ValueError:
                    # Skip directories that are not under movies_base.
                    continue

                print(rel_folder)
                if "friends" in str(rel_folder):
                    # Build the corresponding transcript file path.
                    transcript_file = transcripts_base / rel_folder / movie_file.with_suffix('.tsv').name
                else:
                    transcript_file = transcripts_base / rel_folder / f"movie10_{movie_file.with_suffix('.tsv').name}"

                print(f"Movie:      {movie_file}")
                print(f"Transcript: {transcript_file}")

                # Load video frames, audio waveform, and transcript.
                video, audio, transcript, sample_rate, fps_video = None, None, None, None, None
                if modality == 'all' or modality == 'video':
                    video, fps_video = load_video(movie_file, verbose=verbose)
                if modality == 'all' or modality == 'audio' or modality == 'video' or modality == 'transcript':
                    audio, sample_rate = load_audio(movie_file)
                if modality == 'all' or modality == 'transcript':
                    transcript = load_transcript(transcript_file)

                # round fps video
                if fps_video:
                    fps_video = round(fps_video)

                if transcript is not None:
                    transcript = resample_transcript(transcript, interval)
                    
                total_duration = audio.shape[1] / sample_rate
                num_intervals = int(total_duration // interval)

                if verbose:
                    print(f"Total duration: {total_duration:.2f} seconds")
                    print(f"Number of intervals: {num_intervals}")

                # Create the output directory if it doesn't exist.
                output_folder = Path(output_dir) / rel_folder
                output_folder.mkdir(parents=True, exist_ok=True)

                # Create a h5 file to store the features.
                output_file = output_folder / movie_file.with_suffix('.h5').name

                if verbose:
                    print(f"Output file: {output_file}")

                # Create a HDF5 file to store the features.
                with h5py.File(output_file, 'w') as f:
                    features_datasets = {} 
                    # Extract features at each interval.
                    for i in tqdm(range(num_intervals)):
                        video_section, audio_section, transcript_section = extract_section(
                            video, audio, transcript, interval, i, sample_rate, modality, fps_video
                        )

                        # if i == 100:
                        #     video_section_g = video_section
                        #     audio_section_g = audio_section
                        #     transcript_section_g = transcript_section
    
                            
                        #     # convert the video_section from int8 to float32
                        #     video_section = video_section.int()
                        #     # plot the first and the last frame of the video
                        #     plt.imshow(video_section[0].permute(1, 2, 0).cpu().numpy())
                        #     plt.show()
                        #     plt.imshow(video_section[-1].permute(1, 2, 0).cpu().numpy())    
                        #     plt.show()
    
    
                        #     # display the audio section as an html audio element
                        #     torchaudio.save("audio.wav", audio_section.float(), sample_rate)
                        #     from IPython.display import Audio
                        #     Audio("audio.wav")
    
    
                        #     # print the transcript section
                        #     print(transcript_section)
                        
                        #     # break for testing
                        #     assert False
                        
                            
        
                        output_features = extract_fn(video_section, audio_section, transcript_section, verbose)
                        
                        for layer_name, tensor in output_features.items():
                            # Convert the tensor to a numpy array (on CPU) before storing.
                            tensor_np = tensor.cpu().numpy()
                            if layer_name not in features_datasets:
                                # Create a new dataset and initialize it with the first interval's data.
                                features_datasets[layer_name] = f.create_dataset(
                                    layer_name,
                                    data=tensor_np[np.newaxis, ...],
                                    maxshape=(None,) + tensor_np.shape,
                                    dtype=np.float16,
                                    chunks=True,
                                )
                            else:
                                ds = features_datasets[layer_name]
                                ds.resize(ds.shape[0] + 1, axis=0)
                                ds[-1] = tensor_np
                                
                        # if features_dataset is None:
                        #     features_max_shape = (None,) + output_features.shape
                        #     print(features_max_shape, output_features.shape)
                        #     features_dataset = f.create_dataset(
                        #         'features', 
                        #         shape= output_features.unsqueeze(0).shape,
                        #         maxshape=features_max_shape,
                        #         dtype=np.float16,
                        #         chunks=True,    
                        #     )
                        # else:
                        #     features_dataset.resize(features_dataset.shape[0] + 1, axis=0)
                        #     features_dataset[-1] = output_features

def resample_transcript(transcript: pd.DataFrame, new_interval: float) -> pd.DataFrame:
    """
    Pre-aggregates transcript data into new time intervals.
    
    Parameters:
        transcript (pd.DataFrame): DataFrame with columns 'words_per_tr', 'onsets_per_tr', and 'durations_per_tr'.
        new_interval (float): Desired interval in seconds for grouping.
        
    Returns:
        pd.DataFrame: New DataFrame where each row aggregates words whose end time (onset + duration)
                      falls within the same new interval.
    """
    all_words = []
    all_onsets = []
    all_durations = []

    for _, row in transcript.iterrows():
        # Skip rows without valid onsets.
        if not row['onsets_per_tr'] or row['onsets_per_tr'] == []:
            continue

        # Convert string representations if needed.
        onsets = row['onsets_per_tr']
        words = row['words_per_tr']
        durations = row['durations_per_tr']
        if isinstance(onsets, str):
            onsets = ast.literal_eval(onsets)
        if isinstance(words, str):
            words = ast.literal_eval(words)
        if isinstance(durations, str):
            durations = ast.literal_eval(durations)

        all_words.extend(words)
        all_onsets.extend(onsets)
        all_durations.extend(durations)

    # Create a DataFrame with one row per word.
    df = pd.DataFrame({
        'word': all_words,
        'onset': all_onsets,
        'duration': all_durations
    })
    df['word_end'] = df['onset'] + df['duration']
    
    # Determine the new interval index.
    df['new_index'] = (df['word_end'] // new_interval).astype(int)
    
    # Group by the new interval index.
    grouped = df.groupby('new_index').agg({
        'word': list,
        'onset': list,
        'duration': list,
        'word_end': list
    }).reset_index(drop=True)
    
    return grouped


def extract_section(
    video: torch.Tensor,
    audio: torch.Tensor,
    transcript: pd.DataFrame,
    interval: float,
    index: int,
    sample_rate: int,
    modality: str = 'all',
    fps_video: float = 30
) -> Tuple[torch.Tensor, torch.Tensor, List[str]]:
    """
    Extracts a section of audio, video, and transcript data based on the interval and index.

    Parameters:
        audio (torch.Tensor): Tensor containing audio waveform (2, num_samples).
        video (torch.Tensor): Tensor containing video frames (num_frames, 3, 224, 224).
        transcript (pd.DataFrame): DataFrame containing transcript data.
        interval (float): Interval (in seconds) at which to extract features.
        index (int): Index of the interval to extract.
        sample_rate (int): Sample rate of the audio waveform.
        modality (str): Modality to extract features from. Default is 'all'. Options are 'video', 'audio', 'transcript'. The function will return None for the other modalities.

    Returns:
        tuple: (audio_section, video_section, transcript_section) where:
            - audio_section is a torch.Tensor containing the audio data for the section.
            - video_section is a torch.Tensor containing the video data for the section.
            - transcript_section is a list of strings containing the transcript data for the section.
    """
    audio_section = None
    video_section = None
    transcript_section = []

    # Compute the start and end times for the section.
    start_time = index * interval
    end_time = (index + 1) * interval

    if modality == 'all' or modality == 'audio':
        # Extract audio data for the section.
        audio_start = int(start_time * sample_rate)
        audio_end = int(end_time * sample_rate)
        audio_section = audio[:, audio_start:audio_end]

    if modality == 'all' or modality == 'video':
        # Extract video data for the section.
        # fps_video = 30 # 29.97
        frame_start = round(start_time * fps_video)
        frame_end = round(end_time * fps_video)
        video_section = video[frame_start:frame_end]

    if modality == 'all' or modality == 'transcript':
        transcript_section = transcript['word'].iloc[index]
                    
        print(transcript_section)

    return video_section, audio_section,  transcript_section



### Load the model and experiment to see it's inputs and layers

In [10]:
from transformers import WhisperModel, AutoProcessor, pipeline
from datasets import load_dataset

os.environ["HF_DATASETS_OFFLINE"] = "1"
os.environ["HF_HUB_OFFLINE"] = "1"


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3" #"openai/whisper-small"

model = WhisperModel.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=False, use_safetensors=True
)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)


audio_test, sampling_rate = load_audio('/scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e01a.mkv', sampling_rate = 16000, stereo = False)
result = processor(audio_test[0][:int(sampling_rate*1.49)], sampling_rate = sampling_rate, return_tensors = 'pt')
with torch.no_grad():
    embeddings = model.encoder.forward(result['input_features'].half().to(device), output_hidden_states = True)

/tmp/ipykernel_3934037/1603866394.py:50: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("ffmpeg")


In [11]:
model.encoder

WhisperEncoder(
  (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
  (embed_positions): Embedding(1500, 1280)
  (layers): ModuleList(
    (0-31): 32 x WhisperEncoderLayer(
      (self_attn): WhisperSdpaAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (activation_fn): GELUActivation()
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((1280,),

In [12]:
layers_to_extract = ["layer_norm", 'layers.31.fc2', 'layers.25.fc2', 'layers.12.fc2']

# Use the feature extractor as a context manager so that hooks are cleaned up automatically.
with HuggingFaceFeatureExtractor(model.encoder, layers_to_extract, detach=True) as extractor:
    with torch.no_grad():
        # Run a forward pass. The extractor clears previous features automatically.
        outputs = extractor(result['input_features'].half().to('cuda'))
    
        # Retrieve the extracted features.
        features = extractor.features
    
        # Print out the shape of the main model output.
        print("Main model output (last_hidden_state) shape:", outputs.last_hidden_state.shape)
    
        # Iterate over the extracted features and print their shapes.
        for layer_name, activation in features.items():
            print(f"Layer: {layer_name}, Feature shape: {activation.shape}")


Main model output (last_hidden_state) shape: torch.Size([1, 1500, 1280])
Layer: layers.12.fc2, Feature shape: torch.Size([1, 1500, 1280])
Layer: layers.25.fc2, Feature shape: torch.Size([1, 1500, 1280])
Layer: layers.31.fc2, Feature shape: torch.Size([1, 1500, 1280])
Layer: layer_norm, Feature shape: torch.Size([1, 1500, 1280])


### Define the extract_fn function

In [13]:
from functools import wraps

# Define the extractor.
extractor = HuggingFaceFeatureExtractor(model.encoder, layers_to_extract, detach=True)

def extract_fn(
    video: torch.Tensor, 
    audio: torch.Tensor, 
    transcript: List[str], 
    verbose: bool
) -> Dict[str, torch.Tensor]:
    # Modify this function using the feature extractor
    # video is a tensor with shape [fps * interval, 3, heigth, width] on fp16 from 0-255
    # audio is a tensor with shape [1 if mono 2 if stereo, sampling_rate * interval] on fp16
    # transcript is list of strings of words.
    
    with torch.no_grad():
        inputs = processor(audio[0][:int(sampling_rate*1.49)], sampling_rate = sampling_rate, return_tensors = 'pt')
        
        # Perform feature extraction here
        outputs = extractor(inputs['input_features'].half().to(device))

    # Retrieve the extracted features.
    features = extractor.features
    dict_return = {}
    for layer_name, activation in features.items():
        # Taking the mean bcz the features are so large
        dict_return[layer_name] = activation.mean(1)
        
    extractor.clear()
    return dict_return

### Start extracting features

In [15]:
# Example usage:
parts = ['s1'] # ['s1', 's2', 's3', 's5', 's6', 's7', 'wolf', 'life', 'bourne', 'figures'] 
movies_base = "/scratch/gpfs/ri4541//algonauts_2025.competitors/stimuli/movies"
transcripts_base = "/scratch/gpfs/ri4541//algonauts_2025.competitors/stimuli/transcripts"
out_dir = '/scratch/gpfs/ri4541/algonauts2025'


extract_features(parts = parts, movies_base = movies_base, transcripts_base = transcripts_base, output_dir = out_dir, extraction_fn = extract_fn, verbose = True, modality = 'audio')

friends/s1
Movie:      /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e14a.mkv
Transcript: /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/transcripts/friends/s1/friends_s01e14a.tsv


/tmp/ipykernel_3934037/1603866394.py:50: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("ffmpeg")


Total duration: 731.88 seconds
Number of intervals: 491
Output file: /scratch/gpfs/ri4541/algonauts2025/friends/s1/friends_s01e14a.h5


100%|██████████| 491/491 [00:15<00:00, 32.28it/s]


friends/s1
Movie:      /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e23b.mkv
Transcript: /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/transcripts/friends/s1/friends_s01e23b.tsv
Total duration: 687.50 seconds
Number of intervals: 461
Output file: /scratch/gpfs/ri4541/algonauts2025/friends/s1/friends_s01e23b.h5


...

100%|██████████| 495/495 [00:15<00:00, 32.49it/s]


friends/s1
Movie:      /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e04b.mkv
Transcript: /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/transcripts/friends/s1/friends_s01e04b.tsv
Total duration: 749.06 seconds
Number of intervals: 502
Output file: /scratch/gpfs/ri4541/algonauts2025/friends/s1/friends_s01e04b.h5


100%|██████████| 502/502 [00:15<00:00, 32.29it/s]


friends/s1
Movie:      /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e22a.mkv
Transcript: /scratch/gpfs/ri4541/algonauts_2025.competitors/stimuli/transcripts/friends/s1/friends_s01e22a.tsv
Total duration: 706.06 seconds
Number of intervals: 473
Output file: /scratch/gpfs/ri4541/algonauts2025/friends/s1/friends_s01e22a.h5


100%|██████████| 473/473 [00:14<00:00, 32.43it/s]


In [ ]:
# import h5py
# import numpy as np

# # Path to the generated h5 file.
# file_path = '/kaggle/working/friends/s1/friends_s01e21b.h5'

# with h5py.File(file_path, 'r') as f:
#     print("Datasets in the file:")
#     for layer_name in f.keys():
#         dataset = f[layer_name]
#         print(f"Layer: {layer_name}")
#         print(f" - Shape: {dataset.shape}")
#         print(f" - Data type: {dataset.dtype}")
        
#         num_intervals_to_print = min(5, dataset.shape[0])
#         for i in range(num_intervals_to_print):
#             # Get the i-th interval data, flatten it, and print the first 5 elements.
#             interval_data = dataset[i]
#             flat_data = interval_data.flatten()
#             first_five = flat_data[:5] if flat_data.size >= 5 else flat_data
#             print(f" Interval {i}: first 5 elements: {first_five}")
#         print("-" * 50)
